# 前提条件

| 項目   | 内容                           |
|------|------------------------------|
| 概要   | 顧客満足度アンケートの Data             |
| 対象期間 | 2019年１月 ~ ４月                 |
| 保存方法 | Database                     |
| 目的   | 顧客満足度の向上                     |
| 変数   | アンケートの取得日, Comment, 満足度（５段階） |

In [2]:
import pandas as pd

survey = pd.read_csv('sample_code/chapter_10/survey.csv')
print(len(survey))
survey.head()

86


,datetime,comment,satisfaction
0,2019/3/11,駅前に若者が集まっている(AA駅),1
1,2019/2/25,スポーツできる場所があるのが良い,5
2,2019/2/18,子育て支援が嬉しい,5
3,2019/4/9,保育園に入れる（待機児童なし）,4
4,2019/1/6,駅前商店街が寂しい,2


In [13]:
survey.isna().sum()

datetime        0
comment         2
satisfaction    0
dtype: int64

Comment に２つ欠損値を確認。
欠損値を除去する。

In [14]:
survey = survey.dropna()
survey.isna().sum()

datetime        0
comment         0
satisfaction    0
dtype: int64

不要な文字を除去する。

In [9]:
survey['comment'] = survey['comment'].str.replace('AA', '')
survey['comment'].head()

0     駅前に若者が集まっている(駅)
1    スポーツできる場所があるのが良い
2           子育て支援が嬉しい
3     保育園に入れる（待機児童なし）
4           駅前商店街が寂しい
Name: comment, dtype: object

括弧内が補足的な情報なので除去する。※正規表現を利用

In [13]:
survey['comment'] = survey['comment'].str.replace('\(.+?\)', '', regex=True)
survey.head()

,datetime,comment,satisfaction
0,2019/3/11,駅前に若者が集まっている,1
1,2019/2/25,スポーツできる場所があるのが良い,5
2,2019/2/18,子育て支援が嬉しい,5
3,2019/4/9,保育園に入れる（待機児童なし）,4
4,2019/1/6,駅前商店街が寂しい,2


全角の括弧が残っているの除去する

In [15]:
survey['comment'] = survey['comment'].str.replace('（.+?\）', '', regex=True)
survey.head()

,datetime,comment,satisfaction
0,2019/3/11,駅前に若者が集まっている,1
1,2019/2/25,スポーツできる場所があるのが良い,5
2,2019/2/18,子育て支援が嬉しい,5
3,2019/4/9,保育園に入れる,4
4,2019/1/6,駅前商店街が寂しい,2
